In [ ]:
import logging
logging.basicConfig(level=logging.WARNING)
logger = logging.getLogger(__name__)

from dotenv import load_dotenv
# lädt die (versteckte) Datei die im selben Ordner liegt wie dieses notebook file
# die Datei muss .env benannt sein, diese Dateien sind in der Regel ausgeblendet, 
# weil Dateien mit . andeuten, dass diese im Explorer ausgeblendet werden.
# In der Datei
load_dotenv()

In [ ]:
import os
from pydantic_ai import Agent
from pydantic import BaseModel, Field 
from pydantic_ai.models.google import GoogleModel
from pydantic_ai.models.openai import OpenAIModel
from pydantic_ai.providers.openai import OpenAIProvider
from pydantic_ai.providers.google import GoogleProvider
from pydantic_ai.providers.azure import AzureProvider
from enum import Enum

# Sentiment Analyse

In [ ]:
class Sentiment(str, Enum):
    POSITIVE = "POSITIVE"
    NEGATIVE = "NEGATIVE"
    NEUTRAL = "NEUTRAL"

class SentimentExtractor(BaseModel):
    sentiment: Sentiment

In [ ]:
# Ein GPT Modell via Azure
model = OpenAIModel(
    'gpt-4o',
    provider=AzureProvider(
        azure_endpoint=os.environ["AZURE_ENDPOINT"],
        api_version='2025-01-01-preview',
        api_key=os.environ["AZURE_KEY"],
    ),
)

agent = Agent(model, output_type=[SentimentExtractor])

In [ ]:
result = await agent.run("Das Wetter heute ist durchwachsen")
print(result.output.sentiment)

In [ ]:
result = await agent.run("Das Wetter heute ist super schön")
print(result.output.sentiment)

In [ ]:
result = await agent.run("Das Wetter heute ist eisig, nur Regen und kalter Wind")
print(result.output.sentiment)

## Aspekt-orientierter Sentiment Analysis
Häufig genug sind Sätze aber etwas kompliziter und mehrere ASPEKTE werden erwähnt
die eine Wertung erhalten (ein SENTIMENT)
Wie man die bearbeitet, schauen wir uns jetzt an

In [ ]:
class AspektSentiment(BaseModel):
    target: str = Field(..., description="Das Objekt oder Ziel einer wertenden Aussage")
    attribute: str = Field(..., description="Die Eigenschaft eines Objektes auf die Bezug genommen wird")
    sentiment: Sentiment

class AspektSentimentExtractor(BaseModel):
    aspects: list[AspektSentiment]   

In [ ]:
aspect_agent = Agent(model, output_type=[AspektSentimentExtractor])

In [ ]:
result = await aspect_agent.run("Das neue Handy ist okay, die Kamera ist fantastisch aber das Display ist was dunkel und der Akku ist schwach")

In [ ]:
for aspect in result.output.aspects:
    print(aspect)